<h3> Imports </h3>

In [ ]:
import numpy as np
import pandas as pd
import ebooklib
import re
import nltk
from nltk.util import ngrams
from ebooklib import epub
from bs4 import BeautifulSoup
from nltk.util import ngrams
import spacy

<h3>Reading the epub file : </h3>

In [ ]:
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

In [ ]:
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]

In [ ]:
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

In [ ]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

In [ ]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [ ]:
#epub file as a list
out=epub2text('The-Art-of-Digital-Marketing-The-Definitive-Guide-to-Creating-Strategic-Targeted-and-Measurable-Online-Campaigns.epub')

<h3> Data Munging </h3>

In [ ]:
#getting a string out of the text
text_epub = ' '.join(out)

In [ ]:
#saving the text in a newfile
with open("text.cvs", "w") as text_file:
    print(f"text: {text_epub}", file=text_file)

In [ ]:
#reading each line
with open("text.cvs", "r") as text:
    text = text.readlines()

In [ ]:
#removing newlines(\n)
text = [l.strip('\n') for l in text]

In [ ]:
#removing numbers
def filter_numbers(string):
    return re.sub(r'\d+', '', string)

text = [filter_numbers(l) for l in text]

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
text = [w for w in text if not w in stop_words]

In [ ]:
#list into string
text = ' '.join(text)

In [ ]:
#tokenizing the text per sentence 
text = re.compile('[.!?] ').split(text)

In [ ]:
#removing punctuation
def filter_punct(string):
    return re.sub(r'[^\w\s]', '', string)

text = [filter_punct(l) for l in text]

In [ ]:
#removing empty space
text = [l for l in text if l]

In [ ]:
#lower casing every word
text = [l.lower() for l in text]

In [ ]:
#using spacy
nlp = spacy.load("en_core_web_sm")
text = ''.join(text)
doc = nlp(text)

In [ ]:
#deleting stop words
text = [token.text for token in doc if not token.is_stop]

In [ ]:
# Lemmatizing 
def lemmatizer(text):        
    sent = []
    for word in text:
        sent.append(word.lemma_)
    return " ".join(sent)
#lemmatizer(doc)

In [ ]:
#3grams
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]
  
#extract_ngrams(text,3)

In [ ]:
#from spacy.lang.en import English

#with open("skills.cvs") as skills:
    #skills.read()

#nlp = English()
#doc = nlp(doc)

#from spacy.matcher import PhraseMatcher

#matcher = PhraseMatcher(nlp.vocab)

#[nlp(skills) for skill in skills]
#patterns = list(nlp.pipe(skills))
#matcher.add("skills", None, *patterns)

#matches = matcher(doc)
#print([doc[start:end] for match_id, start, end in matches])